In [ ]:
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('vader_lexicon')
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import yfinance as yf

# Function to get historical stock data
def get_stock_data(symbol, start_date, end_date):
    stock = yf.download(symbol, start=start_date, end=end_date)
    return stock['Adj Close']

# Function to scrape S&P 500 companies and their symbols from Wikipedia
def get_sp500_companies():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    
    companies = []
    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        symbol = columns[0].text.strip()
        name = columns[1].text.strip()
        companies.append({'Symbol': symbol, 'Company_Name': name})

    return companies

# Sample data - replace this with actual S&P500 company symbols and names
s_and_p_500_data = get_sp500_companies()[:100]  # Limiting to the first 100 companies

# Sort companies alphabetically
s_and_p_500_data.sort(key=lambda x: x['Company_Name'])

# Define the date range for historical stock data
start_date = '2000-01-01'
end_date = '2024-01-01'

# Initialize a DataFrame for the analysis
analysis_data = pd.DataFrame(index=pd.date_range(start_date, end_date))


# Iterate through S&P500 symbols
for i in range(len(s_and_p_500_data)):
    symbol = s_and_p_500_data[i]['Symbol']
    company_name = s_and_p_500_data[i]['Company_Name']

    # Get historical stock data
    stock_data = get_stock_data(symbol, start_date, end_date)

    # Calculate daily percentage change for momentum
    analysis_data[f'{symbol}_Momentum'] = stock_data.pct_change()

    # Handle NaN values in momentum by replacing them with the mean
    mean_momentum = analysis_data[f'{symbol}_Momentum'].mean()
    analysis_data[f'{symbol}_Momentum'] = analysis_data[f'{symbol}_Momentum'].fillna(mean_momentum)

    # Sentiment Analysis using NLTK's VaderSentiment
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = [sid.polarity_scores('Sample news')['compound'] for _ in range(len(analysis_data))]

    # Add sentiment scores to the DataFrame
    analysis_data[f'{symbol}_Sentiment'] = sentiment_scores

    # Print sentiment and momentum scores
    print(f"{symbol} - Sentiment: {analysis_data[f'{symbol}_Sentiment'].unique()}, Momentum: {analysis_data[f'{symbol}_Momentum'].unique()}")

    # Define the target variable based on closing price change
    closing_price_change = stock_data.diff().dropna()  # Calculate difference between consecutive days
    analysis_data[f'{symbol}_Target'] = pd.cut(closing_price_change, bins=[float('-inf'), 0, float('inf')], labels=[-1, 1])

    # Convert the categorical variable to numeric
    analysis_data[f'{symbol}_Target'] = pd.to_numeric(analysis_data[f'{symbol}_Target'], errors='coerce')

    # Calculate the percentage of certainty for each decision
    analysis_data[f'{symbol}_Certainty'] = (
        analysis_data[f'{symbol}_Sentiment'] * analysis_data[f'{symbol}_Momentum']
    ).fillna(0)  # Replace NaN values with 0 for simplicity

# Combine sentiment, momentum, and certainty into a single DataFrame (analysis_data)
for i in range(len(s_and_p_500_data)):
    symbol = s_and_p_500_data[i]['Symbol']
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    
    # Calculate Sentiment and Momentum
    sentiment = calculate_sentiment(stock_data)
    momentum = calculate_momentum(stock_data)

    # Add Sentiment and Momentum to the analysis_data DataFrame
    analysis_data[f'{symbol}_Sentiment'] = sentiment
    analysis_data[f'{symbol}_Momentum'] = momentum

    # Define the target variable based on closing price change
    closing_price_change = stock_data.iloc[-1]['Close'] - stock_data.iloc[0]['Close']
    analysis_data[f'{symbol}_Target'] = pd.cut(closing_price_change, bins=[float('-inf'), 0, float('inf')], labels=[-1, 1])

    # Calculate Certainty
    analysis_data[f'{symbol}_Certainty'] = (
        analysis_data[f'{symbol}_Sentiment'] * analysis_data[f'{symbol}_Momentum']
    ).fillna(0)  # Replace NaN values with 0 for simplicity

# Combine company symbols, targets, and certainties into a single DataFrame
target_data = pd.DataFrame({
    'Symbol': [s_and_p_500_data[i]['Symbol'] for i in range(len(s_and_p_500_data))],
    'Decision': [int(pd.to_numeric(analysis_data[f"{s_and_p_500_data[i]['Symbol']}_Target"]).fillna(0).iloc[-1]) for i in range(len(s_and_p_500_data))],
    'Certainty': [analysis_data[f"{s_and_p_500_data[i]['Symbol']}_Certainty"].iloc[-1] for i in range(len(s_and_p_500_data))]
})

# Display or export the target_data DataFrame with decisions and certainties
print(target_data)

# Save the combined DataFrame to an Excel file
excel_file_path = 'stock_analysis_results.xlsx'
target_data.to_excel(excel_file_path, index=True)
print(f"Excel file created: {excel_file_path}")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/skd/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[*********************100%%**********************]  1 of 1 completed


MMM - Sentiment: [0.], Momentum: [ 0.0003632  -0.03973589  0.02896604 ...  0.00582738  0.00873644
 -0.00337312]


[*********************100%%**********************]  1 of 1 completed


AOS - Sentiment: [0.], Momentum: [ 0.00080645 -0.01440945 -0.00292419 ...  0.0037875   0.00097365
  0.00243194]


[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed
/var/folders/px/1fx__yfn2k74kgwgnv32tkbw0000gn/T/ipykernel_22243/2576113641.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  analysis_data[f'{symbol}_Momentum'] = stock_data.pct_change()
/var/folders/px/1fx__yfn2k74kgwgnv32tkbw0000gn/T/ipykernel_22243/2576113641.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  analysis_data[f'{symbol}_Sentiment'] = sentiment_scores
/var/folders/px/1fx__yfn2k74kgwgnv32tkbw0000gn/T/ipykernel_22243/2576113641.py:74: Perfor

BMY - Sentiment: [0.], Momentum: [ 0.00026142 -0.07565463 -0.00640651 ... -0.00466474  0.00019525
  0.00175718]


[*********************100%%**********************]  1 of 1 completed
/var/folders/px/1fx__yfn2k74kgwgnv32tkbw0000gn/T/ipykernel_22243/2576113641.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  analysis_data[f'{symbol}_Momentum'] = stock_data.pct_change()
/var/folders/px/1fx__yfn2k74kgwgnv32tkbw0000gn/T/ipykernel_22243/2576113641.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  analysis_data[f'{symbol}_Sentiment'] = sentiment_scores
/var/folders/px/1fx__yfn2k74kgwgnv32tkbw0000gn/T/ipykernel_22243/2576113641.py:74: Perfor

AVGO - Sentiment: [0.], Momentum: [ 0.00150542  0.01545153 -0.02799759 ... -0.00505347 -0.00333876
 -0.00548822]


[*********************100%%**********************]  1 of 1 completed